In [2]:
!pip install mlflow

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 14.6 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.9 MB/s eta 0:00:011
     |████████████████████████████████| 145 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 170 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 56 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 52 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 2.2 MB/s eta 0:00:01
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.2-py3-none-any.whl size=17397 sha256=b6b10cad6474b6f0dda5d5dc85046ad0845d5077ed9a71148a867d728c2b5110
  Stored in di

You should consider upgrading via the '/Users/weugene/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
import os
import sys
import json
import warnings
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

MLFLOW_SERVER_URL = 'http://127.0.0.1:5000/'

warnings.filterwarnings("ignore")
np.random.seed(40)
data = pd.read_csv("mlflow-example/wine-quality.csv")

train, test = train_test_split(data)

train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


def generate_submission():  # Report geenration
    client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
    
    runs = []
    models = [
        {'name': m.name, 
         'versions': [
             {'current_stage': v.current_stage, 'run_id': v.run_id, 'status': v.status} 
             for v in m.latest_versions if m.name == 'coursera-top-model']} 
        for m in client.search_registered_models()
    ]
    for e in client.list_experiments():
        if e.name == 'experiment2':
            for run_info in client.list_run_infos(e.experiment_id):
                run = mlflow.get_run(run_info.run_id)
                runs.append({'params': run.data.params, 'status': run.info.status, 'run_id': run.info.run_id})

    with open('submission.json', 'w') as f:
        json.dump({'runs': runs, 'models': models}, f)


mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'experiment2'
mlflow.set_experiment(experiment_name)

# run the experiment
with mlflow.start_run():
    alpha = 0.65
    l1_ratio = 0.45

    # model
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # metrics
    predicted_qualities = lr.predict(test_x)
    rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))
    mae = mean_absolute_error(test_y, predicted_qualities)
    r2 = r2_score(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # save the metric valuse
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)
run_info = client.list_run_infos(experiment.experiment_id)[-1]

print(experiment)
print(run_info)

FileNotFoundError: File b'mlflow-example/wine-quality.csv' does not exist

In [4]:
!mlflow ui

[2021-09-05 14:43:36 +0300] [8893] [INFO] Starting gunicorn 20.1.0
[2021-09-05 14:43:36 +0300] [8893] [INFO] Listening at: http://127.0.0.1:5000 (8893)
[2021-09-05 14:43:36 +0300] [8893] [INFO] Using worker: sync
[2021-09-05 14:43:36 +0300] [8896] [INFO] Booting worker with pid: 8896
^C
[2021-09-05 14:44:36 +0300] [8893] [INFO] Handling signal: int
[2021-09-05 14:44:36 +0300] [8896] [INFO] Worker exiting (pid: 8896)
